In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
kosdaq_ticker = '^KQ11'
kosdaq_data = yf.Ticker(kosdaq_ticker)
kosdaq_hist = kosdaq_data.history(period="max")

kosdaq_hist.reset_index(drop=False, inplace=True)
# kosdaq_hist= kosdaq_hist.stack().reset_index(name='Value')
kosdaq_hist['Date'] = pd.to_datetime(kosdaq_hist['Date']).dt.strftime('%Y-%m-%d')
kosdaq_hist['Date'] = pd.to_datetime(kosdaq_hist['Date'])

print(kosdaq_hist.head())

In [ ]:
# 특정 날짜의 데이터만 필터링하여 출력
specific_date = '2024-04-02'
filtered_data = kosdaq_hist.loc[kosdaq_hist.index == specific_date]
print(filtered_data)

In [ ]:
#https://www.bok.or.kr/portal/main/contents.do?menuNo=200096
fund = pd.read_csv('./data/kbf.csv')

fund

In [ ]:
kbf_fund = fund.melt(id_vars=['통계표', '계정항목', '단위', '변환'], var_name='Date', value_name='KBF')
kbf_fund = kbf_fund.drop(columns=['통계표', '계정항목', '단위', '변환'])
kbf_fund['Date'] = kbf_fund['Date'].str.replace('/', '-').astype('datetime64[ns]')
kbf_fund['Date'] = pd.to_datetime(kbf_fund['Date'])

kbf_fund


In [ ]:
kosdak_fund_data = pd.merge(kosdaq_hist, kbf_fund, on='Date', how='left')
kosdak_fund_data

In [ ]:
import plotly.graph_objects as go


# 캔들스틱 차트 생성
candlestick = go.Candlestick(x=kosdak_fund_data['Date'],
                             open=kosdak_fund_data['Open'],
                             high=kosdak_fund_data['High'],
                             low=kosdak_fund_data['Low'],
                             close=kosdak_fund_data['Close'],
                             name='Kosdak')

#line_chart = go.Scatter(x=kosdak_fund_data['Date'], y=kosdak_fund_data['KBF'], mode='lines', name='Korea Bank Fund', line=dict(color='gray'))

ratio = kosdak_fund_data['Close'] / kosdak_fund_data['KBF']
adjusted_kbf = kosdak_fund_data['KBF'] * ratio.mean() / kosdak_fund_data['Close'].mean()

adjusted_line_chart = go.Scatter(x=kosdak_fund_data['Date'], y=adjusted_kbf*kosdak_fund_data['Close'].mean()/ratio.mean(), mode='lines', line=dict(color='darkblue'), yaxis='y2', name = 'korea bank fund')



# 그래프 레이아웃 설정
layout = go.Layout(title='KOSDAQ Candlestick Chart with Korea Bank Fund', title_x=0.5,
                   xaxis=dict(title='Date'),
                   yaxis=dict(title='Price', side='left'),
                   yaxis2=dict(title='Korea Bank Fund', side='right', overlaying='y', showgrid=False),
                   legend=dict(x=0.85, y=1.1),
                   width=1200, 
                   height=800)



fig = go.Figure(data=[candlestick, adjusted_line_chart], layout=layout)


periods = [('2002-03-25', '2004-08-05'), ('2007-10-10', '2008-11-05'), ('2020-02-27', '2020-04-01')]

# 각 기간에 대해 투명한 회색 박스를 추가
for start, end in periods:
    fig.add_vrect(x0=start, x1=end, 
                  fillcolor='rgba(128,128,128,0.35)',  # 회색에 20% 투명도
                  line=dict(color='rgba(0,0,0,0)', width=0))

fig.show()